In [ ]:
import pandas as pd

In [ ]:
file = "cards/data/171218_Inhalte_Gesamtdatenbank_14.xlsx"

In [ ]:
df = pd.read_excel(file)

In [ ]:
df

In [ ]:
bildmotiv, _ = SkosConceptScheme.objects.get_or_create(dc_title="Bildmotive")
    

In [ ]:
for i, row in df.iterrows():
    temp_cardcol, _ = CardCollection.objects.get_or_create(abbreviation = row["Serien-kürzel"])
    card, _ = Card.objects.get_or_create(title=row['Bildquelle'])
    card.card_collection = temp_cardcol
    card.legacy_id = row['Lfd. Nr.']
    if pd.isnull(row['Nr (Serie)']):
        pass
    else:
        card.number=(row['Nr (Serie)'])
    if pd.isnull(row['Text Bildseite (Zitat)']):
        pass
    else:
        card.text_front = row['Text Bildseite (Zitat)']
    if pd.isnull(row['Text Adressseite (Zitat)']):
        pass
    else:
        card.text_back = row['Text Adressseite (Zitat)']
    if pd.isnull(row['Ausgabe-datum']):
        pass
    else:
        card.published_written = row['Ausgabe-datum']
    if pd.isnull(row['gelaufen / Datum']):
        pass
    else:
        card.run_written = row['gelaufen / Datum']
    if pd.isnull(row['Formale Anmerkungen']):
        pass
    else:
        card.note = row['Formale Anmerkungen']
    if pd.isnull(row['Inhaltliche Anmerkungen']):
        pass
    else:
        card.note_content = row['Inhaltliche Anmerkungen']
    try:
        persons = row['Künstler (Bild)'].split(';')
        for x in persons:
            temp_pers, _ = Person.objects.get_or_create(written_name = x)
            temp_pers.name = x.split(',')[0]
            try:
                temp_pers.forename = x.split(',')[1]
            except IndexError:
                pass
            temp_pers.save()
            card.artist.add(temp_pers)
    except AttributeError:
        pass
    place = row['Index, Orte (nur anhand der Aufschriften)']
    if pd.isnull(place):
        pass
    else:
        temp_place, _ = Place.objects.get_or_create(place_written_name=place)
        card.mentioned_place.add(temp_place)
    try:
        motiv = row['Index, Bildmotive'].split(',')
    except AttributeError:
        motiv = None
    if motiv:
        for x in motiv:
            if x == "Bildmotiv":
                pass
            else:
                temp_bildmotiv, _ = SkosConcept.objects.get_or_create(
                    pref_label=x.strip(),
                    pref_label_lang='ger'
                )
                temp_bildmotiv.scheme.set([bildmotiv])
                temp_bildmotiv.save()
                card.bild_motiv.set([temp_bildmotiv])
    card.save()